In [106]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [107]:
train = pd.read_csv('C:/Users/mahak/OneDrive/Documents/Analytics Vidhya Hackathon/JanataHack - E-Commerce Analytics ML Hackathon/train.csv')
test = pd.read_csv('C:/Users/mahak/OneDrive/Documents/Analytics Vidhya Hackathon/JanataHack - E-Commerce Analytics ML Hackathon/test.csv')

## Train Data

In [108]:
train.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,2015-12-14 18:11,2015-12-14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female
1,u10253,2016-12-14 14:35,2016-12-14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male
2,u19037,2001-12-14 15:58,2001-12-14 15:58,A00002/B00001/C00020/D16944/,female
3,u14556,2023-11-14 2:57,2023-11-14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female
4,u24295,2017-12-14 16:44,2017-12-14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male


In [109]:
#Extracting the number of products 
def numproducts(prd):
    if ';' in prd:
        products=prd.split(';')
        noprds=len(products)
    else:
        prt=prd.split('/')
        noprds=len(prt)
    return(noprds)

In [110]:
new_col = ('TotalProducts')      
new_col_lst = train['ProductList'].apply(lambda x: numproducts(x))    

In [111]:
List=pd.DataFrame(new_col_lst)
df1=List.reset_index()
df=df1.reset_index()
dat=df.drop(['level_0','index'],axis=1)
dat.head()

,ProductList
0,4
1,7
2,5
3,3
4,2


In [112]:
#data_df=dat.rename(columns = {'ProductList':'Total_Products'},axis=1, inplace = True) 
dat.rename({'ProductList': 'Total_Products'}, axis=1, inplace=True)
dat.head()

,Total_Products
0,4
1,7
2,5
3,3
4,2


In [113]:
newdata = pd.concat([train, dat], axis=1)
df_new=newdata.drop(['session_id','startTime','endTime','ProductList'],axis=1)
df_new.head()

,gender,Total_Products
0,female,4
1,male,7
2,female,5
3,female,3
4,male,2


In [114]:
#Handling the datetime column using pandas
train['startTime'] = pd.to_datetime(train['startTime'])
train['endTime'] = pd.to_datetime(train['endTime'])
train['weekday'] = train['startTime'].dt.dayofweek
train['year'] = train['startTime'].dt.year
train['month'] = train['startTime'].dt.month

In [115]:
df_new1=train.drop(['session_id','startTime','endTime','ProductList','gender'],axis=1)

In [116]:
newdata1 = pd.concat([df_new, df_new1], axis=1)
newdata1.head()

,gender,Total_Products,weekday,year,month
0,female,4,0,2015,12
1,male,7,2,2016,12
2,female,5,4,2001,12
3,female,3,1,2023,11
4,male,2,3,2017,12


In [117]:
newdata1['gender'].value_counts() #imbalanced dataset

female    8192
male      2308
Name: gender, dtype: int64

## Applying Machine Learning Model

In [118]:
X = newdata1.drop('gender',axis=1)
y = newdata1['gender']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=34)

In [120]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred_rm = clf.predict(X_test)
print(accuracy_score(y_test,y_pred_rm))

0.780952380952381


In [121]:
#Extracting the num of products from the test data
def numproducts(prd):
    if ';' in prd:
        products=prd.split(';')
        noprds=len(products)
    else:
        prt=prd.split('/')
        noprds=len(prt)
    return(noprds)

new_col = ('TotalProducts')      
new_col_lst = test['ProductList'].apply(lambda x: numproducts(x))  

List=pd.DataFrame(new_col_lst)
df1=List.reset_index()
df=df1.reset_index()
dat=df.drop(['level_0','index'],axis=1)

dat.rename({'ProductList': 'Total_Products'}, axis=1, inplace=True)

newdata = pd.concat([test, dat], axis=1)
df_test=newdata.drop(['session_id','startTime','endTime','ProductList'],axis=1)

In [122]:
test['startTime'] = pd.to_datetime(test['startTime'])
test['endTime'] = pd.to_datetime(test['endTime'])
test['weekday'] = test['startTime'].dt.dayofweek
test['year'] = test['startTime'].dt.year
test['month'] = test['startTime'].dt.month
df_tt2=test.drop(['session_id','startTime','endTime','ProductList'],axis=1)
newdata2 = pd.concat([df_test, df_tt2], axis=1)
newdata2.head()

,Total_Products,weekday,year,month
0,5,6,2008,12
1,5,5,2019,12
2,5,4,2001,12
3,4,6,2008,12
4,5,0,2015,12


In [123]:
pred = clf.predict(newdata2)
pred

array(['female', 'female', 'female', ..., 'female', 'female', 'female'],
      dtype=object)

In [124]:
list = pred
print("Length of original list",len(list))

distinct_list= (Counter(list).keys())
print("List with distinct elements:\n",distinct_list)
print("Length of distinct list:",len(distinct_list))

Length of original list 4500
List with distinct elements:
 dict_keys(['female', 'male'])
Length of distinct list: 2


In [125]:
test['session_id'] = test
test['gender'] = pred
test = test[['session_id', 'gender']]

In [126]:
print("Before OverSampling, counts of label 'female': {}".format(sum(y_train == 'female'))) 
print("Before OverSampling, counts of label 'male': {} \n".format(sum(y_train == 'male')))

Before OverSampling, counts of label 'female': 5715
Before OverSampling, counts of label 'male': 1635 



In [127]:
from imblearn.over_sampling import SMOTE
from collections import Counter
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train,y_train)

In [128]:
print("After OverSampling, counts of label 'female': {}".format(sum(y_res == 'female'))) 
print("After OverSampling, counts of label 'male': {} \n".format(sum(y_res == 'male')))

After OverSampling, counts of label 'female': 5715
After OverSampling, counts of label 'male': 5715 



In [129]:
clf_sm = RandomForestClassifier()
clf_sm.fit(X_res, y_res)
y_pred_rm = clf_sm.predict(X_test)
print(accuracy_score(y_test,y_pred_rm)) # accuracy is not improved after oversampling the minority class

0.5053968253968254


In [130]:
#test.to_csv('final_submission.csv', index=False)